In [2]:
import numpy as np
from scipy.special import jv
from numba import jit,autojit

In [25]:
@autojit
def func(n_max,d,e,in_w):
    out_w = np.multiply(d,in_w)
    j = 2
    for i in range(int(n_max-2)):
        out_w[i] = out_w[i] - e[i] * in_w[j]
    i =1
    for j in range(1,int(n_max)-1):
        out_w[j] = out_w[j] + e[i] * in_w[j]
    return out_w

In [26]:
n_max = 100
d = np.random.rand(n_max)*1e55
e = np.random.rand(n_max-1)
in_w = np.zeros(n_max, dtype = complex)
for i in range(len(in_w)):
    in_w[i] = np.complex(np.random.rand(),np.random.rand()) 

In [27]:
%timeit func(n_max,d,e,in_w)

The slowest run took 26725.73 times longer than the fastest. This could mean that an intermediate result is being cached 
1 loops, best of 3: 18.4 µs per loop
